In [1]:
import tensorflow as tf
import numpy as np
slim = tf.contrib.slim
import sys
sys.path.insert(0, "/media/allen/mass/tflite_development")
sys.path.insert(0, "/media/allen/mass/tflite_development/models/research/slim")

W0911 17:47:56.962924 140676850853632 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
import mobilenet_v1
import nets.mobilenet.mobilenet_v2 as mobilenet_v2
batch_size = 1
height, width = 224, 224
num_classes = 1

inputs = tf.placeholder(dtype=np.float32, shape=(1,224,224,3), name='input')
logits, end_points = mobilenet_v2.mobilenet(inputs, num_classes, is_training=False)
output = logits - logits * -1
# logits, end_points = mobilenet_v1.mobilenet_v1(inputs, num_classes, is_training=False)

W0911 17:47:58.193070 140676850853632 deprecation_wrapper.py:119] From /media/allen/mass/tflite_development/models/research/slim/nets/mobilenet/mobilenet.py:397: The name tf.nn.avg_pool is deprecated. Please use tf.nn.avg_pool2d instead.

W0911 17:47:58.199522 140676850853632 deprecation_wrapper.py:119] From /media/allen/mass/tflite_development/models/research/slim/nets/mobilenet/mobilenet.py:364: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



In [3]:
graph = tf.get_default_graph()

In [4]:
for i in graph.as_graph_def().node:
    print("{:45} {:20}".format(i.name, i.op))

input                                         Placeholder         
MobilenetV2/input                             Identity            
MobilenetV2/Conv/weights/Initializer/random_uniform/shape Const               
MobilenetV2/Conv/weights/Initializer/random_uniform/min Const               
MobilenetV2/Conv/weights/Initializer/random_uniform/max Const               
MobilenetV2/Conv/weights/Initializer/random_uniform/RandomUniform RandomUniform       
MobilenetV2/Conv/weights/Initializer/random_uniform/sub Sub                 
MobilenetV2/Conv/weights/Initializer/random_uniform/mul Mul                 
MobilenetV2/Conv/weights/Initializer/random_uniform Add                 
MobilenetV2/Conv/weights                      VariableV2          
MobilenetV2/Conv/weights/Assign               Assign              
MobilenetV2/Conv/weights/read                 Identity            
MobilenetV2/Conv/dilation_rate                Const               
MobilenetV2/Conv/Conv2D                       Conv2

In [5]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [6]:
train_writer = tf.summary.FileWriter(".")
train_writer.add_graph(sess.graph)

In [6]:
model_name = "mobilenet_v2_init_sub"
# output_name = "MobilenetV2/Predictions/Reshape_1"
output_name = "sub"
saver = tf.train.Saver()
saver.save(sess, "./output/{}.ckpt".format(model_name))

'./output/mobilenet_v2_init_sub.ckpt'

In [7]:
with tf.io.gfile.GFile('./output/{}.pb'.format(model_name), 'wb') as f:
    f.write(sess.graph_def.SerializeToString())

In [8]:
from tensorflow.python.tools.freeze_graph import freeze_graph
d = freeze_graph("./output/{}.pb".format(model_name), "",
             True, "./output/{}.ckpt".format(model_name),
             output_name, 'save/restore_all',
             'save/Const:0', "./output/frozen_{}.pb".format(model_name),
             True, '')

W0911 17:48:21.387048 140676850853632 deprecation.py:323] From /home/allen/.pyenv/versions/tf/local/lib/python2.7/site-packages/tensorflow/python/tools/freeze_graph.py:127: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0911 17:48:22.263156 140676850853632 deprecation.py:323] From /home/allen/.pyenv/versions/tf/local/lib/python2.7/site-packages/tensorflow/python/tools/freeze_graph.py:233: convert_variables_to_constants (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
W0911 17:48:22.264735 140676850853632 deprecation.py:323] From /home/allen/.pyenv/versions/tf/local/lib/python2.7/site-packages/tensorflow/python/framework/graph_util_impl.py:270: extract_sub_graph (from tensorflo

### use pretrain weight

In [2]:
with tf.gfile.GFile('/home/allen/multitaskTensorflow.pb', 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

tf.import_graph_def(
        graph_def, 
        input_map=None, 
        return_elements=None, 
        name="", 
        op_dict=None, 
        producer_op_list=None)

graph = tf.get_default_graph()

W0906 15:53:32.590431 140486433224448 deprecation.py:506] From <ipython-input-2-bdbfeb44b362>:11: calling import_graph_def (from tensorflow.python.framework.importer) with op_dict is deprecated and will be removed in a future version.
Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.


In [3]:
for i in graph_def.node:
    print("{:45} {:20}".format(i.name, i.op))

data                                          Placeholder         
conv1/weights                                 Const               
conv1/weights/read                            Identity            
conv1/Pad/paddings                            Const               
conv1/Pad                                     Pad                 
conv1/Conv2D                                  Conv2D              
batch_norm1/scale                             Const               
batch_norm1/scale/read                        Identity            
batch_norm1/offset                            Const               
batch_norm1/offset/read                       Identity            
batch_norm1/mean                              Const               
batch_norm1/mean/read                         Identity            
batch_norm1/variance                          Const               
batch_norm1/variance/read                     Identity            
batch_norm1/batch_norm1/add/y                 Const           

conv37/Conv2D_217                             Conv2D              
conv37/Conv2D_218                             Conv2D              
conv37/Conv2D_219                             Conv2D              
conv37/Conv2D_220                             Conv2D              
conv37/Conv2D_221                             Conv2D              
conv37/Conv2D_222                             Conv2D              
conv37/Conv2D_223                             Conv2D              
conv37/Conv2D_224                             Conv2D              
conv37/Conv2D_225                             Conv2D              
conv37/Conv2D_226                             Conv2D              
conv37/Conv2D_227                             Conv2D              
conv37/Conv2D_228                             Conv2D              
conv37/Conv2D_229                             Conv2D              
conv37/Conv2D_230                             Conv2D              
conv37/Conv2D_231                             Conv2D          

In [5]:
img = graph.get_tensor_by_name('data:0')
# embedding = graph.get_tensor_by_name('prefix/bn2/bn2/add_1:0')
fc3 = graph.get_tensor_by_name('fc3/fc3:0')
fc2 = graph.get_tensor_by_name('fc2/fc2:0')
fc4 = graph.get_tensor_by_name('fc4/fc4:0')

In [6]:
tflite_model = tf.lite.toco_convert(graph_def, [img], [fc2, fc3, fc4])
open("/media/allen/mass/tflite_development/multitaskTensorflow.tflite", "wb").write(tflite_model)

W0906 15:54:27.293814 140486433224448 deprecation.py:323] From <ipython-input-6-e90205729e8e>:1: toco_convert (from tensorflow.lite.python.convert) is deprecated and will be removed in a future version.
Instructions for updating:
Use `lite.TFLiteConverter` instead.
